# Loading the Peft Library

In [ ]:
!pip install -q peft datasets accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM

# Loading the model and the tokenizers.

In [ ]:
model_name = "HuggingFaceTB/SmolLM2-360M"
NUM_VIRTUAL_TOKENS = 35  # These no. of tokens will be added in the pretrained model, which will be trainable

NUM_EPOCHS_PROMPT = 10
NUM_EPOCHS_CLASSIFIER = 10
device = "cuda"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # Add this line to set the padding token

foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map = device
)

# Inference with the pre trained Smollm2 model

In [ ]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        #temperature=0.2,
        #top_p=0.95,
        #do_sample=True,
        repetition_penalty=1.5, #Avoid repetition.
        early_stopping=True, #The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

I will run the same sentence on both models in order to compare the pre-trained model with the same model following the prompt-tuning procedure.

In [ ]:
import os
from datasets import load_dataset
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit
from transformers import TrainingArguments




In [ ]:
input_classifier = tokenizer("Sentence : Because everyone knows this islam men is the devil. Label : ", return_tensors="pt")
foundational_outputs_prompt = get_outputs(foundational_model,
                                          input_classifier.to(device),
                                          max_new_tokens=3)

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

The model is generating ambiguous response and it completes the sentence as best as it can.

In [ ]:
dataset_classifier = "SetFit/ethos_binary"

def concatenate_columns_classifier(dataset):
    def concatenate(example):
        example['text'] = "Sentence : {} Label : {}".format(example['text'], example['label_text'])
        return example

    dataset = dataset.map(concatenate)
    return dataset

In [ ]:
data_classifier = load_dataset(dataset_classifier)
data_classifier['train'] = concatenate_columns_classifier(
    data_classifier['train'])

data_classifier = data_classifier.map(
    lambda samples: tokenizer(samples["text"]),
    batched=True)
train_sample_classifier = data_classifier["train"].remove_columns(
    ['label', 'label_text', 'text'])

In [ ]:
data_classifier

In [ ]:
train_sample_classifier

In [ ]:
print(train_sample_classifier[2:3])

# Prompt-Tuning Configuration

In [ ]:
generation_config_classifier = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,   #This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text="Indicate if the text contains hate speech or no hate speech.",
    #Number of virtual tokens to be added and trained.
    num_virtual_tokens=NUM_VIRTUAL_TOKENS,   # 20 virtual tokens
    #The pre-trained model.
    tokenizer_name_or_path=model_name
)

In [ ]:
peft_model_classifier = get_peft_model(
    foundational_model,
    generation_config_classifier)
print(peft_model_classifier.print_trainable_parameters())  # Get the total percentage of trainable parameters

**Did you notice the decrease in trainable parameters?** That's incredible. <br>**0.0093% of the available paramaters will be trained.**

In [ ]:
import os

working_dir = "/content/Prompt_Tuning_SmolLM2/"

#Is best to store the models in separate folders.
#Create the name of the directories where to store the models.
output_directory_classifier =  os.path.join(working_dir, "peft_outputs_classifier")

#Just creating the directoris if not exist.
if not os.path.exists(working_dir):
    os.mkdir(working_dir)

In [ ]:
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate=0.0035, epochs=6, autobatch=True):
    training_args = TrainingArguments(
        output_dir=path, # Where the model predictions and checkpoints will be written
        #use_cpu=True, # This is necessary for CPU clusters.
        auto_find_batch_size=autobatch, # Find a suitable batch size that will fit into memory automatically
        learning_rate= learning_rate, # Higher learning rate than full fine-tuning
        #per_device_train_batch_size=4,
        num_train_epochs=epochs,
        report_to="none"
    )
    return training_args

In [ ]:
training_args_classifier = create_training_arguments(
    output_directory_classifier,
    3e-2,
    NUM_EPOCHS_CLASSIFIER)

# Training the Model

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling
def create_trainer(model, training_args, train_dataset):
    trainer = Trainer(
        model=model, # We pass in the PEFT version of the foundation model, SmolLM2
        args=training_args, #arguments
        train_dataset=train_dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=False indicates not to use masked language modeling
    )
    return trainer

In [ ]:
trainer_classifier = create_trainer(peft_model_classifier,
                                   training_args_classifier,
                                   train_sample_classifier)
trainer_classifier.train()

# Inference Model

In [ ]:
trainer_classifier.model.save_pretrained(output_directory_classifier)

In [ ]:
from peft import PeftModel

loaded_model_peft = PeftModel.from_pretrained(foundational_model,
                                         output_directory_classifier,
                                         #device_map=device,
                                         is_trainable=False)

In [ ]:
loaded_model_peft.load_adapter(output_directory_classifier, adapter_name="classifier")
loaded_model_peft.set_adapter("classifier")

In [ ]:
input_classifier = tokenizer("Sentence : Because everyone knows this islam men is the devil.  Label : ", return_tensors="pt").to(device)

loaded_model_sentences_outputs = get_outputs(loaded_model_peft,
                                             input_classifier,
                                             max_new_tokens=3)
print(tokenizer.batch_decode(loaded_model_sentences_outputs, skip_special_tokens=True))